A good tool makes simple things easy and complex things possible.

If you don't know which solution is better, start with the one that's simpler.

In [1]:
import os
from yaml import load, CLoader as Loader

data = load(open('env/local-env.yml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})

from hemlock import push_app_context

push_app_context()

<Flask 'hemlock.app'>

In [ ]:
from hemlock import Check, Input, Page, Label, Range, Select, Validate

Page(
    Input('<p>Please enter your date of birth in mm-dd-yyyy format.</p>'),
    Validate.require(Check(
        '<p>Please indicate your gender.</p>',
        ['Male', 'Female', 'Other']
    )),
    Check(
        '<p>Please indicate your ethnicity. Check as many as apply.</p>',
        [
            'White',
            'Black or African-American',
            'Asian',
            'Native Hawaiian or other Pacific Islander',
            'Other',
        ],
        multiple=True
    ),
    Select(
        '<p>Please select your current marital status.</p>',
        [
            'Married',
            'Widowed',
            'Divorced',
            'Separated',
            'Never married',
        ]
    ),
    Range(
        '''
        <p>At the right end of the scale are the people who are the 
        best off; those who have the most money, the most 
        education, and the best jobs. On the left are the people 
        who are the worst off; those who have the least money, the 
        least education, and the worst jobs (or are unemployed). 
        Please indicate where you think you stand on this scale.</p>
        ''',
        min=0, max=10
    ),
).preview('Ubuntu')

In [2]:
from hemlock import Input, Validate

inpt = Validate.require(Input('<p>You must respond to this question.</p>'))
inpt

<Input 1>

In [3]:
inpt.validate_functions

[<Validate 1>]

In [4]:
not inpt.response and not inpt.error

True

In [5]:
inpt._validate()
inpt.error

'Please respond to this question.'

In [8]:
inpt.response = 'hello world'
inpt._validate()
inpt.error

''

In [10]:
from datetime import datetime

@Validate.register
def validate_date_format(inpt):
    try:
        datetime.strptime(inpt.response, '%m/%d/%Y')
    except:
        return '<p>Make sure to format your date of birth as mm/dd/yyyy.</p>'
    
Validate.validate_date_format(inpt)
inpt.response = 'I, George Thaddeus Thatch the Third, was born in the first fortnight of August 1792.'
inpt._validate()
inpt.error

'Make sure to format your date of birth as mm/dd/yyyy.'

In [12]:
@Validate.register
def my_func(question):
    # validation logic here
    pass
    
inpt = Validate.my_func(Input())
inpt.validate_functions[0].func

<function __main__.my_func(question)>